In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import stan

try:
    import arviz as az
except ImportError as err:
    print("Please, install arviz for easy visualization of Stan models.")

import nest_asyncio
nest_asyncio.apply()

## Bulding a simple model

Let's start with the simplest problem of all: Bayesian estimation of the mean and variance from a sample of data points. Our model is:
$$
\begin{align}
    \mu &\sim \mathcal{N}(0, 3)\\
    \sigma^2 &\sim \text{Inv-Gamma}(1, 1)\\
    y_n &\sim \mathcal{N}(\mu, \sigma^2) \quad \text{for} \,\, n = 1,\dots,N
\end{align}
$$

In [4]:
### Simulate data
mu = 2.5
sigma = 3
N = 50
y = np.random.normal(mu, sigma, size=N)

In [5]:
### Create data dictionary
data_dict = {
    'y': y,
    'N': N
}

In [6]:
program_code = """

data {
    int<lower=1> N;
    vector[N] y;
}

parameters {
    real mu;
    real<lower=0> sigma2;
}

model {
    mu ~ normal(0, 3);
    sigma2 ~ inverse_gamma(1,1);

    // Data model (liklihood)
    for (n in 1:N)
    {
        y[n] ~ normal(mu, sigma2);
    }

}

"""

In [7]:
### Build model
model = stan.build(program_code, data=data_dict)

Building...


Building: Semantic error:   -------------------------------------------------
    13:  model {
    14:      mu ~ normal(0, 3);
    15:      sigma2 ~ inverse_gamma(1,1);
             ^
    16:  
    17:      // Data model (liklihood)
   -------------------------------------------------

Ill-typed arguments to '~' statement. No function 'inverse_gamma_lpdf' was found when looking for distribution 'inverse_gamma'.

ValueError: Semantic error

In [ ]:
### Fit model
fit = model.sample(num_chains=4, num_samples=1000, num_warmup=500)

In [12]:
### Explore raw model outouts
results_df = fit.to_frame()

In [14]:
### Summarize model
az.summary(fit)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu,2.438,0.056,2.334,2.545,0.001,0.001,3217.0,2582.0,1.0
sigma2,8.092,0.231,7.652,8.517,0.004,0.003,3206.0,2432.0,1.0
sigma,2.844,0.041,2.768,2.920,0.001,0.001,3206.0,2432.0,1.0


In [ ]:
### Visual inspection and diagnostics
az.plot_trace(fit, var_names=['mu', 'sigma'], compact=False, legend=True)
plt.tight_layout()

In [ ]:
### Forest plots
az.plot_forest(fit, var_names=['mu', 'sigma'], combined=True)

In [ ]:
### Plot KDEs
ax = az.plot_kde(results_df.mu, 
                 results_df.sigma, hdi_probs=[0.393, 0.86, 0.99])
ax.axvline(mu, color='black', linestyle='--')
ax.axhline(sigma, color='black', linestyle='-.')

## Bayesian regression model

Next, we will build a simple Bayesian regression model of the form:
$$
\begin{align}
    \nonumber \sigma^2 &\sim \text{Inv-Gamma}(1, 1)\\
    \nonumber \alpha &\sim \mathcal{N}(0, 5)\\
    \nonumber \beta &\sim \mathcal{N}(0, 5)\\
\nonumber y_n &\sim \mathcal{N}(\alpha + \beta\,x_n, \sigma^2) \quad \text{for} \,\, n = 1,\dots,N
\end{align}
$$

In [ ]:
#### Your code here